In [ ]:
using Pkg
# Pkg.add("DataFramesMeta")
Pkg.add("TidierData")

In [1]:
using DataFrames,CSV,DataFramesMeta
# using TidierData:select
# using TidierData:@select
# using TidierData
#Pkg.rm(["Parquet","Parquet2","ParquetFiles"])

In [330]:
my_statement = CSV.read("banks-data.csv", DataFrame)
my_statement
# slice_filter(Description::String) Description == "Turk's Head"
println("Find All my spending related with 🍺🍺")
filter(:Description => n -> n == "Turk's Head", my_statement)
filter(:Description => x -> occursin(r"Turk's Head|Topsham", x), my_statement)

Find All my spending related with 🍺🍺


Row,Date,Description,Type,Money In (£),Money Out (£),Balance (£)
,String15,String31,String3,String7,String7,String7
1,02 Oct 23,Turk's Head,DEB,blank,12,"5,741"
2,30 Oct 23,Zettle_*Topsham Br,DEB,blank,5,"3,229"
3,30 Oct 23,Zettle_*Topsham Br,DEB,blank,5,"3,224"
4,30 Oct 23,Zettle_*Topsham Br,DEB,blank,4,"3,194"
5,06 Nov 23,Turk's Head,DEB,blank,6,"4,677"
6,07 Aug 23,Turk's Head,DEB,blank,5,"4,226"
7,07 Aug 23,Turk's Head,DEB,blank,10,"4,206"
8,09 Aug 23,Turk's Head,DEB,blank,2,"4,161"
9,16 Aug 23,Turk's Head,DEB,blank,5,"3,951"


In [8]:
# @select my_statement :Description :"Money In (£)"
@chain my_statement begin
    @select begin
        :Description 
        :Date
    end
    @slice_sample(n=3)
end

Row,Date
,String15
1,13 Nov 23
2,01 Nov 23
3,12 Jun 23


### Perform Type Casting

In [15]:
function replace_blank(var)::Float64
    var = parse(String, var)
    var = replace(var, r"\,|blank" => "")
    var = replace(var, r"^$" => "0")
    var = parse(Float64, var)
    return var
end

@assert replace_blank("12,323") == 12_323
@assert typeof(replace_blank("12,323")) == Float64
@assert map(replace_blank,["12,323", "212"]::Vector{String}) == [12_323, 212]
try
    @assert begin 
        map(replace_blank,["12,323", "212"]::Vector{String7}) == [12_323, 212]
        replace_blank("12,323"::String7) == 12_323
    end
catch e
    println("This function don't like Vector{String7}\n, $e")
end
# replace_blank(my_statement[1,:"Money Out (£)"])

MethodError: MethodError: no method matching parse(::Type{String}, ::String)

Closest candidates are:
  parse(!Matched::Type{P}, ::AbstractString; kwargs...) where P<:FilePathsBase.AbstractPath
   @ FilePathsBase ~/.julia/packages/FilePathsBase/4RrDh/src/path.jl:77
  parse(!Matched::Type{Rational{T}}, ::AbstractString) where T<:Integer
   @ Base rational.jl:98
  parse(!Matched::Type{T}, ::AbstractString) where T<:Complex
   @ Base parse.jl:386
  ...


In [12]:

# using Dates
# function replace_blank(var::String)::Float64
#     var = replace(var, r"\,|blank" => "")
#     var = replace(var, r"^$" => "0"::String)
#     var = parse(Float64, var)
#     return var
# end

statement_schema = Dict(
    "Date" => (Date, x -> Date(x, "dd u yy") + Dates.Year(2000)),
    "Money Out (£)" => (Float64, replace_blank::Function),
    "Money In (£)" => (Float64, replace_blank::Function),
    "Balance (£)" => (Float64, replace_blank::Function)
)

for (col, (expected_type, transformation_fun)) in statement_schema
    actual_type=eltype(my_statement[:,col])
    # @assert callable(transformation_fun)
    if actual_type != expected_type
        my_statement[:,col] = map(x -> transformation_fun(x), my_statement[:,col])
    else 
        println("No need to cast $col to type $expected_type")
    end
end
# test=[]::Vector{Any}

# for (colName; expect_type, fun) in statement_schema 
#     actual_type=eltype(my_statement[:,colName])
#     # test = [test;actual_type==expect_type]
#     # if actual_type==expect_type
        
# end
# test

UndefVarError: UndefVarError: `Date` not defined

In [327]:

# Type Casting using function and `rtransform`
using Dates
function replace_blank(var)::Float64
    var = replace(var, r"\,|blank" => "")
    var = replace(var, r"^$" => "0")
    var = parse(Float64, var)
    return var
end

try
    # run this only once
    @chain my_statement begin
            @rtransform! begin
                :Date = Date(:Date, "dd u yy") + Dates.Year(2000)
                :"Money Out (£)" = replace_blank(:"Money Out (£)")
                :"Money In (£)" = replace_blank(:"Money In (£)")
                :"Balance (£)" = replace_blank(:"Balance (£)")  
            end
    end
    println("Changes Made")
catch e 
    println("Possible already performed type casting")
    println(e)
end

Possible already performed type casting
MethodError(

similar, (2.0, Any), 0x00000000000084b1)


my_statement

In [13]:
using DataFramesMeta
my_statement


UndefVarError: UndefVarError: `my_statement` not defined

### Tidy Verse Framework 

In [368]:
# subset(my_statement, :Description => x -> occursin.(r"Turk's Head|Topsham", x))
using DataFramesMeta

my_statement = CSV.read("banks-data.csv", DataFrame)
df = my_statement
@chain my_statement begin
    @subset occursin.(r"Turk's Head|Topsham", :Description)
    # @select r"£"
    # @select r"£" => replace.("blank"=>"")
    # groupby(:Date)
    # @combine :n=sum(:"Money In (£)")
end

# Select Column with symbol £
select(my_statement, r"£")

## in pipe
using Pipe  
@pipe my_statement |> 
    select(_,r"£")

# Select Column while transform
select(my_statement,"Money Out (£)"=>x->replace(x, "blank"=>""))

select(my_statement,
        # as sayint combine this two into a fector
        ["Money Out (£)", "Money In (£)"].=>x-> replace.(x, r"blank|," => s"")
    )

# but actually multi column selector used by
pound_col=names(my_statement, x->occursin('£', x))
transform!(df, pound_col => ByRow((x...)-> (replace.(x, r"blank|,"=>""))) => pound_col)
# transform!(df, pound_col .=> replace.(x, r"blank|,"=>""); renamecols=false)

Row,Date,Description,Type,Money In (£),Money Out (£),Balance (£)
,String15,String31,String3,String,String,String
1,02 Oct 23,STGCOACH/CTYLINK,DEB,,2,5770
2,02 Oct 23,PRET A MANGER,DEB,,5,5764
3,02 Oct 23,STGCOACH/CTYLINK,DEB,,2,5762
4,02 Oct 23,COSTA COFFEE 43011,DEB,,8,5754
5,02 Oct 23,Turk's Head,DEB,,12,5741
6,02 Oct 23,PRET A MANGER,DEB,,5,5735
7,02 Oct 23,CO-OP GROUP 070527,DEB,,11,5724
8,02 Oct 23,London Cocktail Cl,DEB,,23,5700
9,02 Oct 23,EXETER CITY COUNCI,DD,,5,5695


### Base Data Frame Method

In [344]:
my_statement = CSV.read("banks-data.csv", DataFrame)
using StatsBase
# if you put ! on other row you have in place transformation
# which I found in this instance you are better off using base
println("Begain type conversion")# in reactive context it may worth configure code in this format 
# just so it is easy to debug if you run into the same problem twice.

pound_col=names(my_statement, x->occursin('£', x))
# Is x a instance of a copy? 
x=my_statement[!,pound_col]

x=replace.(x,"blank"=>"0")
x=replace.(x,","=>"")
x=parse.(Float64,x)

if x===my_statement[!,pound_col]
    println("Object is the same")
else 
    println("Object is no longer the same")
    println("So you have to do this:")
    vec = [i for i in 1:size(df)[1]]
    println(my_statement[sample(vec,3),:])
    println("See how $pound_col is still stored in string?")
    
    my_expr= quote
        my_statement[!,pound_col]=x
    end
    
    println("you have to do this \n`$(my_expr)`\n")

    eval(my_expr)
    println(my_statement[sample(vec,3),:])
    println("Now the $pound_col is transformed into Float.")
end
# my_statement

Begain type conversion
Object is no longer the same
So you have to do this:
3×6 DataFrame
 Row │ Date       Description         Type     Money In (£)  Money Out (£)  Balance (£) 
     │ String15   String31            String3  String7       String7        String7     
─────┼──────────────────────────────────────────────────────────────────────────────────
   1 │ 12 Jul 23  PRET A MANGER       DEB      blank         8              3,316
   2 │ 08 Aug 23  168 ORIENTAL SUPER  DEB      blank         5              4,178
   3 │ 19 Jun 23  CO-OP GROUP 070527  DEB      blank         22             3,455
See how ["Money In (£)", "Money Out (£)", "Balance (£)"] is still stored in string?
you have to do this 
`begin
    #= /Users/frankliang/Dropbox/Personal/code/Pet Project/12-06 Julia II/Spending.ipynb:26 =#
    my_statement[!, pound_col] = x
end`

3×6 DataFrame
 Row │ Date       Description         Type     Money In (£)  Money Out (£)  Balance (£) 
     │ String15   String31            String3 

Although assign data frame to a variable is considered pointer instance. Pointing at 
a subset is not.

In [345]:
println("I'm a transformed data")
my_statement[sample(vec,3),:]

I'm a transformed data


Row,Date,Description,Type,Money In (£),Money Out (£),Balance (£)
,String15,String31,String3,Float64,Float64,Float64
1,31 Jul 23,FULL FIBRE LIMITED,BGC,2197.0,0.0,4828.0
2,24 Nov 23,NEXT RETAIL LTD,DEB,0.0,16.0,3649.0
3,29 Sep 23,MARKS&SPENCER PLC,DEB,0.0,17.0,5772.0
